# Simulated beam pattern

In [12]:
import numpy as np
import pandas as pd
import healpy as hp
from pygsm import GlobalSkyModel

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

from power_to_temperature import *
from Tgsm import*

import os
import glob
import copy

#for image storage
impath = "images/"

if not os.path.exists(impath):
    os.makedirs(impath)

## Pre-processing beam pattern data and getting GSM maps.

In [13]:
def hibiscus(pathtbeam = 'HIBiscus/', output = "antenna_beam_biscus/"):
    '''
    DESCRIPTION: Process the beam pattern of the HIBiscus antenna and puts the 
    separated hdf5 files in the "antenna_beam" folder. Each frequency
    has an individual file e.g. "050MHz.hdf5"
    
    INPUT: pathbeam = path of folder with beam pattern .txt files
    '''
    
    Freqs = np.arange(50,91)
    for f in Freqs:
        infile = pathtbeam+"0%dMHz.txt"%f
        Data = np.loadtxt(infile)
        df = pd.DataFrame(data = Data,columns = ['Theta','Phi','dB']) 
        df.to_hdf(output + '0%dMHz.hdf5'%f,'df')
        
def mango_peel(file = "mango_peel_beam_pattern.dat", output = "antenna_beam_mango/"):
    '''
    DESCRIPTION: Process the beam pattern of the Mango Peel or the HIBiscus antenna and puts the 
    separated hdf5 files in the "antenna_beam" folder. Each frequency
    has an individual file e.g. "050MHz.hdf5"
    
    INPUT: file = location of beam pattern .dat file
    '''
    
    df = pd.read_csv(file, sep=", ", 
                 index_col=0, header=None, engine = "python") 
    
    new_header = df.iloc[0] 
    df = df[1:]
    df.columns = new_header
    df.reset_index(inplace=True)
    df.rename(columns = {0:"THETA"}, inplace = True)
    
    theta = list(df["THETA"])
    phi = list(df["'PHI'"])
    
    Freq = np.arange(30000000.0, 150000000.0, 1000000)

    for f in Freq:
        data = df[f]

        file = pd.DataFrame(list(zip(theta, phi, data)), columns =['Theta', 'Phi', 'dB'])

        file["Theta"] = pd.to_numeric(file["Theta"], downcast="float")
        file["Phi"] = pd.to_numeric(file["Phi"], downcast="float")
        file["dB"] = pd.to_numeric(file["dB"], downcast="float")

        name = "0" + str(int(f/(1e+6))) + "MHz"
        file.to_hdf(output+name+'.hdf5','df')

In [14]:
if not os.path.exists('antenna_beam_mango'):
    os.makedirs('antenna_beam_mango')
    mango_peel()

if not os.path.exists('antenna_beam_biscus'):
    os.makedirs('antenna_beam_biscus')
    hibiscus()
    
# Loads the GSM maps files in the folder "gsm_maps"
if not os.path.exists('gsm_maps'):
    os.makedirs('gsm_maps')
        
    Freqs = np.arange(50,91) #Frequency of interest 50-90 MHz.
    for f in Freqs:
        gsm = GlobalSkyModel()
        gsm_map = gsm.generate(f)
        DATA = pd.DataFrame(gsm_map)
        DATA.to_hdf('gsm_maps/gsm_%dMHz.hdf5'%f,'df')

## Generate simulated mock antenna pattern

In [15]:
Freqs = np.arange(50,91)
kB = 1.38064852e-23 #Boltzmann constant

# Inverse transformation from Temp to dBms
#Transformacion de dBm's a potencia
P_dBm = lambda source: 30.+10*np.log10(source)


def T2dBm(T, freqs, Bwidth):
    T = np.array(T, dtype=np.float64)
    
    area = 1.0      # m^2
    angle = 55.0    #degrees
    theta = deg2arcsec(angle)
    freq = freqs * 1e6 #Hz
    wavelength = (c / freq) * 100.  # cm
    
    flux_Jy = T*theta**2/(1.36*wavelength**2.) #mJy
    flux_Jy = flux_Jy / 1e3 # Jy
    flux = flux_Jy / 1e26  # Jy
    power = 0.5*flux*area/Bwidth
    dBm = P_dBm(power)
    return dBm

In [16]:
for antenna in ['mango', 'biscus']:
    
    if not os.path.exists('mock_pattern_'+antenna):
        os.makedirs('mock_pattern_'+antenna)

        for Freq in Freqs:    
            Data = pd.read_hdf('antenna_beam_'+antenna+'/0%dMHz.hdf5'%Freq)

            theta,phi = np.radians(Data.values[:,0]),np.radians(Data.values[:,1])
            X,Y,Z = np.sin(theta)*np.cos(phi),np.sin(theta)*np.sin(phi),np.cos(theta)
            dB = Data.values[:,2]
            Temp = Radio_source_trans(dB,Freq,1e6)
            mock_temp = max(Temp)*np.exp(-0.5*(X/0.3)**2.-0.5*(Y/0.3)**2.)
            mock_dBm = T2dBm(mock_temp,Freq,1e6)
            mock_data = np.transpose([Data.values[:,0],Data.values[:,1],mock_dBm])

            df = pd.DataFrame(data = mock_data,columns = ['Theta','Phi','dB']) 
            df.to_hdf('mock_pattern_'+antenna+'/0%dMHz.hdf5'%Freq,'df')

## Generate simulated data (convolution) 

This part of the notebook generates an HDF5 table with the convolved temperatures of the antenna for a day of observation, the binnage in this case is 5 minutes, it can be changed by modifying the bins parameter.

The function to use is T_gsm from Tgsm.py. 

In [22]:
# You can change the path to anywhere your antenna beam pattern files are.

T_gsm('2013-06-14 00:00:00',PATH='mock_pattern_biscus/',OUTPUT='mock_biscus/') 
T_gsm('2013-06-14 00:00:00',PATH='mock_pattern_mango/',OUTPUT='mock_mango/')

T_gsm('2013-06-14 00:00:00', PATH = 'antenna_beam_biscus/', OUTPUT = "biscus") 
T_gsm('2013-06-14 00:00:00', PATH = 'antenna_beam_mango/', OUTPUT = "mango")

KeyboardInterrupt: 

## Diurnal Variation and Simulated Temperature

Checking the diurnal variation of the temperature for every frequency in the rage 50-90 MHz.

In [ ]:
# Data generated in the convolution.
Data = pd.read_hdf('calibration/Tgsm.hdf5')
Data_mock = pd.read_hdf('mock/Tgsm.hdf5')
Freqs = np.arange(50,91)

Checking the Local Sidereal Time (LST) for the date, this is done in order to plot the diurnal variation of a given frequency in functiong of LST, this could be done in function of UTC, but for comparision with SCI-HI paper.

In [ ]:
check_LST('2013-06-14 00:00:00')

In [ ]:
f=70 #Frequency to check
Temp = Data.loc[f].values
Temp_mock = Data_mock.loc[f].values
Time_ = (np.linspace(0,24,len(Temp))+9.62)%24 # For 2013-06-14 LST = UTC + 9:37 (9.62h)
plt.plot(Time_,Temp_mock,'go',markersize = 3,label='Gaussian')
plt.plot(Time_,Temp,'bo',markersize = 4,label='HIbiscus')
plt.xlabel('LST [Hours]')
plt.ylabel('Temperature [Kelvin]')
plt.xlim(0,24)
plt.grid()
plt.legend()
plt.savefig(impath+'variation_%dMHz.png'%f)

Plotting the diurnal variation for every frequency in the range of 50-90 MHz.

Note: No conversion to LST has be done in this part, so the plot's xlabel is in UTC.

In [ ]:
'''

for f in Freqs:
    Temp = Data.loc[f].values
    Time_ = np.linspace(0,24,len(Temp))
    plt.plot(Time_[::5],Temp[::5],'-o',markersize = 4)
    plt.xlabel('UTC [Hours]')
    plt.ylabel('Temperature [Kelvin]')
    plt.xlim(0,24)
    plt.grid()
    plt.savefig('Imagenes/mockvariation_%dMHz.png'%f)
    plt.close()

'''

Plotting the temperature in function of the frequency for the date given.

In [ ]:
Temp = Data['2013-06-14 08:00:00.000'].values
Temp_mock = Data_mock['2013-06-14 08:00:00.000'].values
plt.xlabel('Frequency [MHz]')
plt.ylabel('Temperature [Kelvin]')
plt.plot(Freqs,Temp,'-b',label='HIbiscus')
plt.plot(Freqs,Temp,'ko', markersize=4)
plt.plot(Freqs,Temp_mock,'-g',label='Gaussian')
plt.plot(Freqs,Temp_mock,'ko', markersize=4)
#plt.xlim(60,90)
#plt.ylim(1900,6500)
plt.legend()
plt.savefig(impath+'FreqvsTemp.png')
plt.show()